# Demo. Decoder Basics

In [ ]:
import stim
import pymatching
import sys
sys.path.append("../QEC-Codes")  # Adjust the path to import local modules
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
from numpy.typing import NDArray
import pandas as pd
import seaborn as sns
import time
from sklearn.metrics import r2_score

In [ ]:
# QASM export setup (similar to surface_code_ler)
import os, sys, importlib
qasm_dir = "./bb_code_qasm"
os.makedirs(qasm_dir, exist_ok=True)
# Make sure we can import the conversion utilities
sys.path.insert(0, os.path.abspath("../../../src"))
import noise_util as ns
importlib.reload(ns)

This demo introduces the basics of decoders, focusing on pymatching and BP+OSD.  

**What does decoder do?**  
The input is the error information/syndromes/detection events, along with the detector error model that assigns probability/weights to each error mechanism. For each sample, the decoder will return a vector that gives the predictions for the logical errors that have happened (for surface code it's just one value because there is only one logical operator, for general codes it should a vector).

### 1. Pymatching for repetition codes

### Overall Analysis  
**LER decreases with increasing code distance and decreasing physical error rate (PER).**  
This behavior is consistent with theoretical expectations: as the code distance increases, more physical errors are required to cause a logical error. Similarly, reducing the physical error rate naturally reduces the probability of uncorrectable faults.

<!-- (2) **The break-even PER appears to be ≈ 1, meaning the repetition code always outperforms a single bit.**  
At first glance, this seems to contradict the classical result that repetition codes only outperform a single bit when \( p < 0.5 \) — since majority vote fails above this threshold. However, the classical result is based on **majority vote decoding**, whereas our experiment uses a **matching decoder** (e.g., `pymatching`), which finds the most likely error consistent with the observed syndrome, and can operate effectively even at high error rates. As a result, it can outperform majority vote and maintain logical error suppression for a much wider range of \( p \), even approaching 1.

(3) **The error threshold appears to be >0.5, meaning that increasing the code distance can suppresses the LER even when the PER is large.**  
In standard fault-tolerance theory, a threshold exists below which increasing the code distance improves logical fidelity, and above which larger codes perform worse. Here, because we are using the powerful **matching decoder** on this simple repetition code, the threshold become very large (>0.5). -->

### Analysis of LER Scaling

In principle, a QEC code of distance $d$ can correct up to $t = \lfloor \frac{d-1}{2} \rfloor$ errors. In the **ideal** case where all these errors are corrected, the leading-order contribution to the LER should be $p^{t+1}$, where $t + 1 = \lceil \frac{d+1}{2} \rceil$ is the weight of uncorrectable errors. In this case, a log-log plot of LER vs $p$ should produce a straight line with slope $t + 1$.

In realistic scenarios, however, decoding is imperfect: some correctable errors of weight ≤ $t$ may still lead to logical failure due to decoder suboptimality. As a result, the effective scaling exponent becomes **less than** $t + 1$.

Therefore, by fitting the log-log curve $\log(\text{LER})$ vs $ \log(p) $, we can estimate the effective suppression exponent. The **closer the slope is to t+1**, the more effective the decoder is at approaching the ideal case. Conversely, the difference between the slope and the ideal case quantifies suboptimal correction performance under the given noise model and decoding strategy. We can see that when d increases, the leading degrees become closer to t than t+1 and haves the tendency to drop even below t, indicating the fact that it becomes more and more difficult to correct all the errors up to weight t.  

The equation $$slope = (d_{eff}+1)/2 $$can give a metric called the "effective distance $d_{eff}$", which should be strictly smaller than the true distance $d$.

Next, we verify the "exponential suppression" of LER in code distance.

In [ ]:
import sinter
from typing import List

This time we try to add the 2Q gate error and measurement error as well.

(1) We kind of see a "threshold behavior": the LER only suppresses as distance increases when PER is below 10.8\%. See where the d = 7 and d = 9 line intersect.  
(2) We also see a "break even" behavior: the LER is belowe PER when p < 10\%, where encoding into repetition code improves the error resilience compared to a bare qubit.

### 2. Pymatching for Surface Codes  
We directly use Sinter to streamline the sampling. This time we include the BPOSD decoder as well.

In [ ]:
from bposdd import BPOSD
import BBcode
# from stimbposdd import SinterDecoder_BPOSD, sinter_decoders

In [ ]:
import subprocess
import time

# Compile the C++ code once before the loop
print("Compiling C++ simulator (MPI)...")
# Adjust the path to your C++ file as needed
cpp_file_path = './bb_code_sim.cpp'
executable_path = './bb_code_sim'

compile_command = [
    "mpicxx",  # Use MPI compiler wrapper
    "-std=c++17",
    "-O3",
    "-I../../../../../", # Include for NWQ-Sim headers
    "-DMPI_ENABLED",
    "-o",
    executable_path,
    cpp_file_path
]
try:
    subprocess.run(compile_command, check=True, capture_output=True, text=True)
    print("Compilation successful.")
except subprocess.CalledProcessError as e:
    print("Compilation failed.")
    print("--- stdout ---")
    print(e.stdout)
    print("--- stderr ---")
    print(e.stderr)

In [ ]:

circuit = "bicycle_bivariate_144_12_12_memory_Z"
distance = 12 
rounds = 12
SHOTS = 1000000


In [ ]:
#Parameters
tasks = []
T1 = 10 ** -4
T2 = 10 ** -4
tau = np.array([8*10**-7, 10**-6, 2*10**-6, 3*10**-6, 4*10**-6, 6*10**-6])
lam = 1/T2 - 1/(2*T1)
p_amp = 1 - np.exp(-tau/T1)
p_phase = 1 - np.exp(-lam*tau)
p_list = (p_amp, p_phase)
base_error = 0.000

bb_code = BBcode.BBcode(
    n=144, k=12, d=12, m=6, l=12,
    A=[[3, 0], [0, 1], [0, 2]],
    B=[[0, 3], [1, 0], [2, 0]],
    shift=[0, 0],
    f=[[0, 0], [1, 0], [2, 0], [3, 0], [6, 0], [7, 0], [8, 0], [9, 0], [1, 3], [5, 3], [7, 3], [11, 3]],
    g=[[1, 0], [2, 1], [0, 2], [1, 2], [2, 3], [0, 4]],
    h=[[0, 0], [0, 1], [1, 1], [0, 2], [0, 3], [1, 3]],
    alpha=[[0, 0], [0, 1], [2, 1], [2, 5], [3, 2], [4, 0]],
    beta=[[0, 1], [0, 5], [1, 1], [0, 0], [4, 0], [5, 2]],
)
print(p_amp)
print(p_phase)
i=0
for p_amp, p_phase in zip(p_amp, p_phase):
    noise_profile = [0,0,0,0]
    circuit = bb_code.build_full_BBcode_circuit(rounds=rounds, noise_profile=noise_profile, observable_type="Z", code_capacity=True)

    #Generate stim circuit
    model = ns.ErrorModel(circuit)
    model.setting_error('Identity', False, f'DEPOLARIZE1({base_error})')
    model.setting_error('Single_qubit', False, f'DEPOLARIZE1({base_error})')
    model.setting_error('Two_qubit', False, f'DEPOLARIZE2({base_error})')
    model.setting_error('Measurement', True, f'PAULI_CHANNEL_1({p_amp/4}, {p_amp/4}, {p_amp/4 + p_phase/2})')
    model.setting_error('Reset', True, f'PAULI_CHANNEL_1({p_amp/4}, {p_amp/4}, {p_amp/4 + p_phase/2})')
    stim_circuit = model.generate_noisy_circuit()
    tasks.append(sinter.Task(circuit=stim_circuit, json_metadata={'d': distance, "trial":i}))

    #Generate stabsim circuit
    model.setting_error('Measurement', True, f'AMPLITUDE_DAMP({p_phase}, {p_amp})')
    model.setting_error('Reset', True, f'AMPLITUDE_DAMP({p_phase}, {p_amp})')
    stab_circuit = model.generate_noisy_circuit()
    qasm_output = ns.stim_to_qasm_with_depolarize_noise(stab_circuit)
    # Inject AMPLITUDE_DAMP around M/RESET in QASM using the model settings
    qasm_output = ns.inject_amplitude_damp(qasm_output, model)
    
    try:
        qasm_path = os.path.join(qasm_dir, f"bb_code_d{distance}_p{i}.qasm")
        with open(qasm_path, "w") as f:
            f.write(qasm_output)
    except Exception as e:
        print(f"Failed to export QASM for p={p_amp}: {e}")
    i+=1


In [ ]:
from concurrent.futures import ProcessPoolExecutor
import bposd_mp
import sys
# Add path to MagicCross-main for GPU decoder utilities
# sys.path.insert(0, os.path.abspath('./MagicCross-main/src'))
import cudaq_qec as qec
from beliefmatching import detector_error_model_to_check_matrices
import multiprocessing as mp
from datetime import datetime

def gpu_decode_worker(gpu_idx, det_chunk, obs_chunk, check_matrix, logicals_matrix, decoder_params, result_queue):
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_idx)
    import numpy as np

    try:
        nvdec = qec.get_decoder('nv-qldpc-decoder', check_matrix, **decoder_params)
        results = nvdec.decode_batch(det_chunk)
        decoded = np.array([e.result for e in results])
        
        errors = 0
        for i in range(decoded.shape[0]):
            ans = (logicals_matrix @ decoded[i] + obs_chunk[i]) % 2
            if ans.any():
                errors += 1
        result_queue.put(errors)
    except Exception as e:
        result_queue.put(f"Error in GPU worker {gpu_idx}: {e}")

def gpu_decode_ler(dem, cpp_det_samples, cpp_obs_flips, gpu_num=4, process_per_gpu=1):
    pcm = detector_error_model_to_check_matrices(dem, allow_undecomposed_hyperedges=True)
    logicals_matrix = pcm.observables_matrix
    check_matrix = np.array(pcm.check_matrix.todense(), dtype=np.uint8)

    decoder_params = {
        'error_rate_vec': pcm.priors,
        'max_iterations': 1000,
        'use_osd': True,
        'osd_method': 3, # OSD-CS
        'osd_order': 10,
        'bp_batch_size': 1000,
        'use_sparsity': True,
    }

    total_shots = len(cpp_det_samples)
    if total_shots == 0:
        return 0.0

    num_decode_workers = gpu_num * process_per_gpu
    chunk_size = (total_shots + num_decode_workers - 1) // num_decode_workers
    
    det_chunks = [cpp_det_samples[i:i + chunk_size] for i in range(0, total_shots, chunk_size)]
    obs_chunks = [cpp_obs_flips[i:i + chunk_size] for i in range(0, total_shots, chunk_size)]

    result_queue = mp.Queue()
    processes = []

    for i in range(len(det_chunks)):
        gpu_idx = i % gpu_num
        p = mp.Process(target=gpu_decode_worker, args=(
            gpu_idx, det_chunks[i], obs_chunks[i], check_matrix, logicals_matrix, decoder_params, result_queue
        ))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

    total_errors = 0
    while not result_queue.empty():
        result = result_queue.get()
        if isinstance(result, int):
            total_errors += result
        else:
            print(result) # Print errors from workers

    return float(total_errors) / float(total_shots)


def run_cpp_experiment(tasks, shots):
    cpp_lers = []
    for i, task in enumerate(tasks):
        p = task.json_metadata.get('pM', 0)
        d = task.json_metadata.get('d', 0)
        print(f"\n--- Running C++ (MPI) for p = {i} ---")

        # Paths
        p_tag = f"{p:.0e}".replace("+", "")
        qasm_file_path = os.path.join(qasm_dir, f"bb_code_d{d}_p{i}.qasm")
        cpp_output_path = os.path.join(qasm_dir, f"measurements_d{d}_p{i}.txt")

        # MPI execution
        num_qubits = task.circuit.num_qubits
        mpi_ranks = max(1, int(os.cpu_count() * 0.8))
        iters = shots

        env = os.environ.copy()
        env.update({
            "OMP_NUM_THREADS": "1",
            "OPENBLAS_NUM_THREADS": "1",
            "MKL_NUM_THREADS": "1",
            "VECLIB_MAXIMUM_THREADS": "1",
            "NUMEXPR_NUM_THREADS": "1",
        })

        run_command = [
            "mpirun", "-np", str(mpi_ranks),
            "./bb_code_sim", str(num_qubits), str(iters), qasm_file_path, cpp_output_path
        ]

        cpp_time = 0.0
        try:
            result = subprocess.run(run_command, check=True, capture_output=True, text=True, env=env)
            for line in result.stdout.strip().split('\n'):
                if "Total C++ simulation time" in line:
                    try:
                        cpp_time = float(line.split(':')[1].strip().replace('s', ''))
                    except (ValueError, IndexError):
                        pass
        except subprocess.CalledProcessError as e:
            print("C++ simulation failed to execute.")
            print("Return code:", e.returncode)
            print("--- stdout ---")
            print(e.stdout)
            print("--- stderr ---")
            print(e.stderr)
            cpp_lers.append(0.0)
            continue

        # Post-process + GPU BPOSD decoding
        cpp_ler = 0.0
        try:
            with open(cpp_output_path, "r") as f:
                measurement_strings = [line for line in f.read().strip().split('\n') if line]

            if measurement_strings:
                measurement_data = np.array(
                    [list(map(int, line.split())) for line in measurement_strings],
                    dtype=np.uint8
                ).astype(bool)

                m2d_converter = task.circuit.compile_m2d_converter()
                cpp_det_samples, cpp_obs_flips = m2d_converter.convert(
                    measurements=measurement_data,
                    separate_observables=True
                )

                dem = task.circuit.detector_error_model(
                    decompose_errors=True,
                    ignore_decomposition_failures=True
                )
                
                decode_start_time = time.perf_counter()
                cpp_ler = gpu_decode_ler(dem, cpp_det_samples, cpp_obs_flips)
                decode_time = time.perf_counter() - decode_start_time
                print(f"GPU Decoding Time: {decode_time:.4f}s")


        except FileNotFoundError:
            print(f"C++ output file not found at {cpp_output_path}. Skipping analysis.")
        except Exception as e:
            print(f"An error occurred during decoding: {e}")


        cpp_lers.append(cpp_ler)
        print(f"C++ LER: {cpp_ler}, Sim Time: {cpp_time}s")
    return cpp_lers


In [ ]:
def run_stim_experiment(tasks, shots):
    stim_lers = []
    for i, task in enumerate(tasks):
        print(f"\n--- Running Stim for p = {i} ---")
        start_time = time.perf_counter()

        # Get detector error model from the circuit
        dem = task.circuit.detector_error_model(
            decompose_errors=True,
            ignore_decomposition_failures=True
        )

        # Sample detection events and logical observables from stim
        sampler = task.circuit.compile_detector_sampler()
        stim_det_samples, stim_obs_flips = sampler.sample(
            shots, separate_observables=True, bit_packed=False
        )
        
        sim_time = time.perf_counter() - start_time
        print(f"Stim Sampling Time: {sim_time:.4f}s")

        # Decode using the GPU decoder
        decode_start_time = time.perf_counter()
        stim_ler = gpu_decode_ler(dem, stim_det_samples, stim_obs_flips)
        decode_time = time.perf_counter() - decode_start_time
        
        stim_lers.append(stim_ler)
        print(f"GPU Decoding Time: {decode_time:.4f}s")
        print(f"Stim LER: {stim_ler}")

    return stim_lers

# --- Run Stim-only experiment ---
stim_logical_error_rates = run_stim_experiment(tasks, SHOTS)
print("\nStim complete.")
print("Stim LERs:", stim_logical_error_rates)

try:
    results_dir = "./results"
    p_amp_arr = 1 - np.exp(-tau / T1)
    lam = 1 / T2 - 1 / (2 * T1)
    p_phase_arr = 1 - np.exp(-lam * tau)
    stim_results_path = os.path.join(results_dir, f"stim_bposd_gpu_results_d{distance}_shots{SHOTS}.txt")
    with open(stim_results_path, "w") as f:
        f.write("# tau\tp_amp\tp_phase\tstim_ler\n")
        for ta, pa, pp, ler in zip(tau, p_amp_arr, p_phase_arr, stim_logical_error_rates):
            f.write(f"{ta}\t{pa}\t{pp}\t{ler}\n")
    print(f"Saved Stim (BP+OSD-GPU) results to {stim_results_path}")
except Exception as e:
    print("Failed to save Stim results:", e)


In [ ]:

# --- Run C++-only experiment ---
# reuse SHOTS from previous cell
cpp_logical_error_rates = run_cpp_experiment(tasks, SHOTS)
print("\nC++ complete.")
print("C++ LERs:", cpp_logical_error_rates)
# ...existing code...

try:
    results_dir = "./results"
    p_amp_arr = 1 - np.exp(-tau / T1)
    lam = 1 / T2 - 1 / (2 * T1)
    p_phase_arr = 1 - np.exp(-lam * tau)
    cpp_results_path = os.path.join(results_dir, f"cpp_bposd_gpu_results_d{distance}_shots{SHOTS}.txt")
    with open(cpp_results_path, "w") as f:
        f.write("# tau\tp_amp\tp_phase\tcpp_ler\n")
        for ta, pa, pp, ler in zip(tau, p_amp_arr, p_phase_arr, cpp_logical_error_rates):
            f.write(f"{ta}\t{pa}\t{pp}\t{ler}\n")
    print(f"Saved C++ (BP+OSD) results to {cpp_results_path}")
except Exception as e:
    print("Failed to save C++ results:", e)

In [ ]:
import os, sys, importlib
qasm_dir = "./bb_code_qasm"
os.makedirs(qasm_dir, exist_ok=True)

# New: directory for saved results
results_dir = "./results"
os.makedirs(results_dir, exist_ok=True)

# Make sure we can import the conversion utilities
sys.path.insert(0, os.path.abspath("../../../src"))


In [ ]:
# --- Plotting Results ---
plt.figure(figsize=(8, 6))

plt.plot(tau/T1, stim_logical_error_rates, 'o-', label='Stim LER (BP+OSD)')
plt.plot(tau/T1, cpp_logical_error_rates, 's--', label='C++ LER (BP+OSD)')
plt.xlabel("Physical Error Rate (p)")
plt.ylabel("Logical Error Rate (LER)")
plt.title(f"LER vs. Physical Error Rate for BB Code (d={distance}) Shots= {SHOTS}")
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which="both", ls="--")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# --- Reload and plot previously saved results ---

# Ensure these parameters match the saved files you want to load
# SHOTS = 50000000
# distance = 12
# T1 = 10**-4
# T2 = 10**-4

try:
    results_dir = "./results"
    stim_results_path = Path(results_dir) / f"stim_bposd_results_d{distance}_shots{SHOTS}.txt"
    cpp_results_path = Path(results_dir) / f"cpp_bposd_results_d{distance}_shots{SHOTS}.txt"

    # Check if files exist
    if not stim_results_path.exists() or not cpp_results_path.exists():
        missing = []
        if not stim_results_path.exists(): missing.append(str(stim_results_path))
        if not cpp_results_path.exists(): missing.append(str(cpp_results_path))
        raise FileNotFoundError("Missing results files: " + ", ".join(missing))

    # Function to load data from a tab-separated file
    def load_results(path):
        data = np.loadtxt(path, comments='#', delimiter='\t')
        # Assuming columns are: tau, p_amp, p_phase, ler
        return data[:, 0], data[:, 3]

    # Load data
    tau_s, stim_ler_s = load_results(stim_results_path)
    tau_c, cpp_ler_c = load_results(cpp_results_path)

    # --- Plotting ---
    plt.figure(figsize=(8, 6))
    plt.plot(tau_s / T1, stim_ler_s, 'o-', label='Stim LER (BP+OSD) [loaded]')
    plt.plot(tau_c / T1, cpp_ler_c, 's--', label='C++ LER (BP+OSD) [loaded]')
    
    plt.xlabel(r"$\tau / T1$ Ratio")
    plt.ylabel("Logical Error Rate (LER)")
    plt.title(f"Loaded LER for BB Code (d={distance}), Shots={SHOTS}")
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(True, which="both", ls="--")
    plt.legend()
    plt.tight_layout()
    plt.show()

except Exception as e:
    print("Failed to load and plot")